In [2]:
# 📦 Librerías necesarias
import pandas as pd
import numpy as np
import pmdarima as pm
from tqdm import tqdm

# 1. Cargar dataset
df = pd.read_csv("datasetwg.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 2. Agregar ventas por producto y mes
df_agg = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 3. Crear una lista única de productos
productos = df_agg['product_id'].unique()

# 4. Inicializar lista de resultados
resultados = []

# 5. Loop por cada producto
for prod in tqdm(productos):
    df_prod = df_agg[df_agg['product_id'] == prod].copy()
    df_prod = df_prod.set_index('periodo').asfreq('MS')  # MS: month start
    df_prod = df_prod.fillna(0)

    serie = df_prod['tn']

    # 6. Ajustar auto_arima
    try:
        modelo = pm.auto_arima(
            serie,
            seasonal=True,
            m=12,
            suppress_warnings=True,
            error_action='ignore'
        )

        # 7. Predecir enero y febrero 2020
        forecast = modelo.predict(n_periods=2)

        # 8. Guardar predicción de febrero
        tn_pred_feb = forecast[1]  # segundo valor
        resultados.append((prod, tn_pred_feb))
    except:
        # Si falla, guardar 0
        resultados.append((prod, 0.0))

# 9. Crear DataFrame de resultados
df_resultado = pd.DataFrame(resultados, columns=['product_id', 'tn_predicho'])

# 10. Exportar a CSV
df_resultado.to_csv("prediccion_feb2020_pmdarima.csv", index=False)
print("✅ Predicciones ARIMA guardadas en prediccion_feb2020_pmdarima.csv")


100%|██████████| 780/780 [1:01:22<00:00,  4.72s/it]

✅ Predicciones ARIMA guardadas en prediccion_feb2020_pmdarima.csv


In [1]:
# 📦 Librerías necesarias
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

# 1. Cargar dataset
df = pd.read_csv("datasetwg.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 2. Agregar ventas por producto y mes
df_agg = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 3. Crear lista de productos únicos
productos = df_agg['product_id'].unique()

# 4. Inicializar resultados
resultados = []

# 5. Loop por producto
for prod in tqdm(productos):
    df_prod = df_agg[df_agg['product_id'] == prod].copy()
    df_prod = df_prod.set_index('periodo').asfreq('MS')
    df_prod = df_prod.fillna(0)
    serie = df_prod['tn']

    mejor_aic = np.inf
    mejor_orden = None
    mejor_estacional = None
    mejor_modelo = None

    # 6. Grid manual simple para SARIMAX: (p,d,q) x (P,D,Q,12)
    for p in range(2):
        for d in range(2):
            for q in range(2):
                for P in range(2):
                    for D in range(2):
                        for Q in range(2):
                            try:
                                modelo = SARIMAX(
                                    serie,
                                    order=(p, d, q),
                                    seasonal_order=(P, D, Q, 12),
                                    enforce_stationarity=False,
                                    enforce_invertibility=False
                                )
                                modelo_fit = modelo.fit(disp=False)
                                aic = modelo_fit.aic
                                if aic < mejor_aic:
                                    mejor_aic = aic
                                    mejor_orden = (p, d, q)
                                    mejor_estacional = (P, D, Q, 12)
                                    mejor_modelo = modelo_fit
                            except:
                                continue

    # 7. Predecir enero y febrero 2020
    if mejor_modelo is not None:
        try:
            forecast = mejor_modelo.forecast(steps=2)
            tn_pred_feb = forecast.iloc[1]  # segundo valor
            resultados.append((prod, tn_pred_feb))
        except:
            resultados.append((prod, 0.0))
    else:
        resultados.append((prod, 0.0))

# 8. Crear DataFrame de resultados
df_resultado = pd.DataFrame(resultados, columns=['product_id', 'tn_predicho'])

# 9. Exportar
df_resultado.to_csv("prediccion_feb2020_sarimax.csv", index=False)
print("✅ Predicciones SARIMAX guardadas en prediccion_feb2020_sarimax.csv")


100%|██████████| 780/780 [1:23:55<00:00,  6.46s/it]

✅ Predicciones SARIMAX guardadas en prediccion_feb2020_sarimax.csv
